In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
%config InlineBackend.figure_formats = ['jpg']

from scipy import interpolate
from scipy import integrate

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
plt.rcdefaults()
plt.rcParams.update({'figure.dpi':100})
#plt.rcParams.update({'figure.autolayout':'True'})
plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'font.family': 'sans-serif'})
plt.rcParams.update({'mathtext.default':'regular'})
plt.rcParams.update({'mathtext.fontset':'stixsans'})
plt.rcParams.update({'axes.linewidth': 1.5})
plt.rcParams.update({'xtick.major.width': 1.25 })
plt.rcParams.update({'xtick.minor.size': 2.5})
plt.rcParams.update({'xtick.minor.width': 1.25 })
plt.rcParams.update({'ytick.major.size': 5})
plt.rcParams.update({'ytick.major.width': 1.25 })
plt.rcParams.update({'ytick.minor.size': 2.5})
plt.rcParams.update({'ytick.minor.width': 1.25 })
plt.rc('legend',**{'fontsize':'x-small'})
#plt.figure(1,figsize = [11.0, 8.5])
plt.rcParams.update({'xtick.direction':'in'})
plt.rcParams.update({'ytick.direction':'in'})
plt.rcParams.update({'xtick.top':'True'})
plt.rcParams.update({'ytick.right':'True'})
#plt.rcParams.update({'axes.formatter.limits': (-2, 2)})
plt.rcParams.update({'lines.markersize': 8}) #6 is the default
plt.rcParams.update({'legend.framealpha': 1.0})
plt.rcParams.update({'legend.fancybox': 'False'})
plt.rcParams.update({'axes.grid':'True'})

In [3]:
# Global variables
Percentages = [0.6827, 0.9545]

In [4]:
#
# Returns log-likelihood
def make_loglike(alpha, beta, gamma, theta):
    

def log_to_like(loglike):
    return np.exp(-0.5*loglike)

# x denotes cosmological parameters, y denotes nuisance parameter
def get_levels(loglike):
    likelihood = log_to_like(loglike)
    ds = np.sort(likelihood.flatten())[::-1]
    cs = np.cumsum(ds)
    cs /= cs[-1]
    return np.array([ds[np.argmax(cs > l)] for l in Percentages])[::-1]

def normalize(loglike, x):
    likelihood = log_to_like(loglike)
    raw_sum = np.trapz(likelihood, x)
    return likelihood/raw_sum

# marginalize over param y for axis = 0,
#                        x for axis = 1
# a denotes x or y
def marginalize(loglike, a, axis):
    likelihood = log_to_like(loglike)
    l_axis = np.sum(likelihood, axis=axis)
    l_normed = normalize(l_axis, a)
    return l_normed

# maximize over param y
def maximize(loglike, x, y, mean_y):
    likelihood = log_to_like(loglike)
    pos = np.where(y==mean_y)[0][0]
    l = likelihood[pos]
    return normalize(l, x)

# take derivatives with respect to y
def get_tk_marg(loglike, y):
    dL = np.gradient(loglike, y, axis=0, edge_order=2)
    ddL = np.gradient(dL, y, axis=0, edge_order=2)

    L_marg = loglike - 0.5*dL*(1/ddL)*dL + np.log(0.5*ddL)

    return normalize(np.exp(-0.5*L_marg)[Y==0], x)



In [17]:
def get_levels(loglike):
    ds = np.sort(np.exp(-0.5*loglike).flatten())[::-1]
    cs = np.cumsum(ds)
    cs /= cs[-1]
    return np.array([ds[np.argmax(cs > l)] for l in Percentages])[::-1]


@interact(alpha=widgets.FloatSlider(min=0, max=10),
          betax= widgets.FloatSlider(min=-5, max=5), 
          gammax=widgets.FloatSlider(min=-5, max=5),
          gammay=widgets.FloatSlider(min=-5, max=5),
          theta= widgets.FloatSlider(min=0, max=np.pi/2))

def plot_like(alpha, betax, gammax, gammay, theta):
    xyrange = (-10,10)
    x_base, y_base = np.meshgrid(*2*[np.linspace(*xyrange, 1000)])
    
    x = x_base*np.cos(theta) + y_base*np.sin(theta)
    y = -x_base*np.sin(theta) + y_base*np.cos(theta)

    loglike = x**2 + y**2 + alpha*(x*y)**2
    likelihood = np.exp(-0.5*loglike)

    plt.gca().set_aspect(1)
    plt.contour(likelihood, extent=2*xyrange, levels=get_levels(loglike))
    plt.xlim((-3,3))
    plt.ylim((-3,3))

interactive(children=(FloatSlider(value=0.0, description='alpha', max=10.0), FloatSlider(value=0.0, descriptio…